# Sudoku with a SAT Solver

This notebook demonstrates solving Sudoku using a boolean satisfiability (SAT) solver.

A SAT problem consists of a number of boolean (`true`/`false`) **_variables_**,
and a number of **_constraints_**. Each constraint is a **_disjunction_** (logical or)
of some number of **_literals_**, where each literal is either a variable or the
logical inverse (not) of a variable.

A solution to the SAT problem is an assignment of `true` or `false` to some number
of the variables that satisfies **_all_** the constraints. The entire problem can
be viewed as a conjunction (logical and) of disjunctions (logical ors). This is called
[**_conjuctive normal form_**](https://en.wikipedia.org/wiki/Conjunctive_normal_form)
or **_CNF_**.

The job of a SAT solver is to (ideally) find all solutions. So it should translate from
a **_conjunction of disjunctions_** (CNF) to a **_disjunction of conjunctions_**, which is
called [**_disjunctive normal form_**](https://en.wikipedia.org/wiki/Disjunctive_normal_form)
or **_DNF_**.

The Rexl SAT solver is a fairly basic one. The core function is
```
Sat.Solve(count, constraints, max_solutions)
```
where
* `count` is the number of literals (not variables), so must be even.
* `constraints` is a sequence of **_clauses_** (constraints).
  * Each clause is a sequence of **_literals_**, representing a **_disjunction_** (logical or)
    of the literals.
  * Each literal is an `I8` (default integer type) value that is at least zero and less than
    `count`. The **_literals_** come in inverse pairs with an even literal, such as `8`, being
    paired with the succeeding odd literal, `9`. The SAT solver maintains that two paired literals
    are inverse, that is, one is `true` and the other is `false`.
* `max_solutions` is the maximum number of solutions that the SAT solver will produce.

The `Sat.Solve` function returns a sequence of **_clauses_** where each clause is again a sequence
of **_literals_**. For the return result, each clause should be interpreted as a **_conjunction_**
of the literals, in contrast to the `constraints` parameter, which is a sequence of clauses where
each clause is a **_disjunction_** of literals.

It is convenient (and traditional) to assign each variable in a CNF an index starting
from zero. The `count` passed to `Sat.Solve` is twice the number of variables. The
literal represented by twice the index of a variable is taken to represent the variable
while the succeeding odd literal is taken to represent the inverse (logical negation) of
the variable. Note: the association of a variable with an even literal in an inverse pair is
entirely arbitrary. Alternatively, one could associate a variable with an odd literal and the
inverse of the variable with the associated even literal. The user of the SAT solver can
choose which convention to use. That is, the SAT solver doesn't know about the **_variables_**,
only about **_literals_**. Which literal corresponds to a variable is immaterial to the SAT solver.

Rexl provides some functions to help build a representation of a CNF. One is
```
Sat.Not(literal)
```
where `literal` is an `I8` value, as described above. This returns the `xor` of the
`literal` with `1`, which is its inverse literal. For example

In [1]:
Sat.Not(3);
Sat.Not(2);

2
3


Rexl also provides
```
Sat.AtMostOne(literals)
```
where `literals` is a sequence of literals, each of which is an `I8` value as described
above. This function returns a sequence of clauses, where each clause is a sequence of
exactly two literals, and each of the literals is the inverse literal of one of the
source literals. The number of clauses produced is the number of pairs taken from `literals`,
which is **_n choose 2_**, if `n` is the number of items in `literals`. 

For example, if there are three logical variables `x`, `y`, and `z` assigned variable indices
`0`, `1`, and `2`, respectively, then:
* The literals `0` and `1` represent `x` and `!x`, respectively.
* The literals `2` and `3` represent `y` and `!y`, respectively.
* The literals `4` and `5` represent `x` and `!x`, respectively.

Calling `Sat.AtMostOne([0, 3, 4])` represents that at most one of `x`, `!y`, `z` should be true.
This is represented by `(!x or y) and (!x or !z) and (y or !z)`, which is represented by
a sequence containing **_3 choose 2_** sequences of literals, namely
```
[ [1, 2], [1, 5], [2, 5] ]
```
This is indeed what `Sat.AtMostOne` returns.

In [2]:
Sat.AtMostOne([0, 3, 4])

Seq<Seq<i8>>
   0) Seq<i8>
         0) 1
         1) 2
   1) Seq<i8>
         0) 1
         1) 5
   2) Seq<i8>
         0) 2
         1) 5


## Define a function to map a digit character to its value.

We define a function that maps from a Unicode character to its "value" from the
perspective of Sudoku. We want to support traditional Sudoku (of rank 3) as
well as higher ranks, up to rank 6, so we need a total of 36 symbols. Traditional
Sudoku uses `"123456789"`. To get 36 symbols, we'll also use `0` and `A`
through `Z`, so our symbols are `"1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"`.

This function should map a Unicode character to its position in our symbol list,
and return `-1` if the Unicode character is not one of our symbols.

In [3]:
// Accepts a Unicode code point. Returns the index into
// "1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ", or -1 if not found.
// Note that 0 follows 9.

func ToDigit(ch) :=
  ch - "1"[0] if "1"[0] <= ch <= "9"[0]      else
  9           if           ch  = "0"[0]      else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  -1;

## Define some functions to invoke the SAT solver.

In [4]:
// pre is a text value containing the pre-filled values. Pass null or empty for none.
// M is the size/rank, with 3 meaning standard Sudoku. Maximum supported value is 6.
// MaxSln is the maximum number of solutions that should be produced.
func Sudoku.Private.Core(pre, M, MaxSln) :=
  With(
    N : M^2,
    NumCells: N^2,
    NumVars : N^3,
    Prefilled: pre,

    Symbols : "1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ",

    // SAT solver variables each have an index from 0 up to (but not including)
    // the number of variables, NumVars. Each (bool) variable represents whether a
    // particular value is placed in a particular cell. The variable index is the
    // cell number times N plus the value.
    Vars : Range(NumVars),

    // Group the variable indices by cell (row and col), by value and row, by value and column,
    // and by value and N x N block.
    VarsByRowCol : Vars->GroupBy(it div N),
    VarsByValRow : Vars->GroupBy([key] it mod N, [key] it div N div N),
    VarsByValCol : Vars->GroupBy([key] it mod N, [key] it div N mod N),
    VarsByValBlk : Vars->GroupBy([key] it mod N, [key] it div N div M mod M, [key] it div N div M div N),

    // Each SAT solver constraint is a disjunction (logical or) of some literals. A literal is either
    // a variable or the logical inverse (not) of the variable. The former is represented by the
    // variable index times two. The latter is one more than that. Consequently, each constraint is a
    // sequence of these numbers, each number being either double a variable index, indicating the variable
    // is in the disjunction, or that plus one, indicating the logical inverse of the variable is in
    // the disjunction.
    Imposed :
      Range(NumCells)
        // Map characters in Prefilled to the value represented by that character.
        ->{cell: it, value: Prefilled[it]->ToDigit()}
        // Filter to only valid values.
        ->TakeIf(0 <= value < N)
        // Compute the corresponding variable indices and multiply be two indicating
        // that the variable should be true. Each such value/literal is wrapped in its
        // own constraint/disjunction/sequence, forcing the variable to be true.
        ->Map([2 * (N * cell + value)]),

    // Construct the normal Sudoku constraints. These don't depend on the imposed
    // cell values.
    Clauses : Chain(
      // Each cell should have a value.
      2 * VarsByRowCol,

      // Sat.AtMostOne takes a sequence of literals and produces a sequence of
      // disjunctions, each disjunction containing the inverse of two of the
      // literals.

      // At most one value per cell.
      // At most one occurrence of a value in a row.
      // At most one occurrence of a value in a column.
      // At most one occurrence of a value in a block.
      VarsByRowCol->ChainMap(Sat.AtMostOne(2 * it)),
      VarsByValRow->ChainMap(Sat.AtMostOne(2 * it)),
      VarsByValCol->ChainMap(Sat.AtMostOne(2 * it)),
      VarsByValBlk->ChainMap(Sat.AtMostOne(2 * it)),

      // The imposed constraints.
      Imposed
    ),

    // The SAT solver produces a sequence of solutions (lazily), not just one.
    // Each solution is a sequence of literals that must all be true (a conjunction).
    // The parameters of Sat.Solve are: the number of literals, the constraint
    // clauses, and the maximum number of solutions to generate.
    Boards : Sat.Solve(2 * NumVars, Clauses, MaxSln)->Map(as sln,
      sln
        // Filter to the true variables, map to variable indices, and sort smallest to largest.
        ->Filter(it mod 2 = 0)->Div(2)->SortUp()
        // Group by row and map each group item to its symbol.
        ->GroupBy(it div N div N, Symbols[it mod N : *1])
        // Glue the row values together.
        ->Map(it->Concat("|"))
        // Glue the rows together.
        ->Concat("\n")
        // Prefix with a newline.
        | "\n" & _
    ),

    // Produce a record containing the clauses, imposed cell values, sequence of boards,
    // and original prefilled cell specification.
    {Clauses, Imposed, Boards, Prefilled}
  );

// Define two additional convenience UDFs, so the caller doesn't have
// to specify M or MaxSln on every call.
func Sudoku.Solve(pre) := Private.Core(pre, 3, 10);
func Sudoku.Solve(maxSln, pre) := Private.Core(pre, 3, maxSln);

## Solve a Puzzle

In [5]:
Results := Sudoku.Solve(
  "....2..7." &
  "....34..." &
  "358......" &
  "5.48....." &
  "...1...89" &
  "..2.....6" &
  "24....7.." &
  ".9...52.." &
  "....671.."
);

Results.Boards->Count();
Results.Boards;

1
Seq<str>
   0) 
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4


### Erase One of the Required Values

The erased value is the `3` in the second row (replaced with `_` to emphasize).
If there is still just one solution, that value wasn't needed to ensure
uniqueness.

Note that the MIP solver form can't easily determine whether there are multiple solutions.
The SAT solver handles this with ease.

In [6]:
Results := Sudoku.Solve(
  "....2..7." &
  "...._4..." &
  "358......" &
  "5.48....." &
  "...1...89" &
  "..2.....6" &
  "24....7.." &
  ".9...52.." &
  "....671.."
);

Results.Boards->Count();
Results.Boards;

7
Seq<str>
   0) 
4|6|1|5|2|9|8|7|3
7|2|9|3|8|4|6|5|1
3|5|8|6|7|1|9|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|4|2|5|8|9
9|8|2|7|5|3|4|1|6
2|4|6|9|1|8|7|3|5
1|9|7|4|3|5|2|6|8
8|3|5|2|6|7|1|9|4
   1) 
4|6|1|3|2|9|8|7|5
7|2|9|5|8|4|6|3|1
3|5|8|6|7|1|9|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|7|4|3|5|1|6
2|4|6|9|1|8|7|5|3
1|9|7|4|3|5|2|6|8
8|3|5|2|6|7|1|9|4
   2) 
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
   3) 
4|6|1|5|2|9|8|7|3
7|2|9|3|8|4|6|5|1
3|5|8|6|7|1|9|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|7|4|3|5|1|6
2|4|6|9|1|8|7|3|5
1|9|7|4|3|5|2|6|8
8|3|5|2|6|7|1|9|4
   4) 
4|6|1|3|2|8|9|7|5
7|2|9|6|5|4|8|3|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|4|2|5|8|9
9|8|2|5|7|3|4|1|6
2|4|6|9|3|1|7|5|8
1|9|7|4|8|5|2|6|3
8|3|5|2|6|7|1|9|4
   5) 
4|6|1|3|2|8|9|7|5
7|2|9|6|5|4|8|3|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|4|2|5|8|9
9|8|2|5|7|3|4|1|6
2|4|6|9|8|1|7|5|3
1|9|7|4|3|5

### Add a Bad Required Value

Add an extra required value that is inconsistent with the solution.
The SAT solver finds no solutions.

Note that the MIP solver form can fill in until no further placements are possible.
The SAT solver can't do this.

In [7]:
Results := Sudoku.Solve(
  "....2..74" &
  "....34..." &
  "358......" &
  "5.48....." &
  "...1...89" &
  "..2.....6" &
  "24....7.." &
  ".9...52.." &
  "....671.."
);

Results.Boards->Count();
Results.Boards;

0
Seq<str>


## Hardest Sudoku

The "internet" claims this is the world's most difficult sudoku puzzle, taken from
[here](https://www.kristanix.com/sudokuepic/worlds-hardest-sudoku.php). The author of it,
Finnish mathematician Arto Inkala, calls it AI Escargot.

In [8]:
Results := Sudoku.Solve(
    "1....7.9." &
    ".3..2...8" &
    "..96..5.." &
    "..53..9.." &
    ".1..8...2" &
    "6....4..." &
    "3......1." &
    ".4......7" &
    "..7...3.."
);

Results.Boards->Count();
Results.Boards;

1
Seq<str>
   0) 
1|6|2|8|5|7|4|9|3
5|3|4|1|2|9|6|7|8
7|8|9|6|4|3|5|2|1
4|7|5|3|1|2|9|8|6
9|1|3|5|8|6|7|4|2
6|2|8|7|9|4|1|3|5
3|5|6|4|7|8|2|1|9
2|4|1|9|3|5|8|6|7
8|9|7|2|6|1|3|5|4


### Erase a Required Value

When the required value `9` near the end of the first row is removed, there are `20`
solutions.

In [9]:
Results := Sudoku.Solve(
    100, // max solutions to show
    "1....7._." &
    ".3..2...8" &
    "..96..5.." &
    "..53..9.." &
    ".1..8...2" &
    "6....4..." &
    "3......1." &
    ".4......7" &
    "..7...3.."
);

Results.Boards->Count();
Results.Boards;

20
Seq<str>
   0) 
1|5|4|8|3|7|6|2|9
7|3|6|9|2|5|1|4|8
8|2|9|6|4|1|5|7|3
4|7|5|3|1|2|9|8|6
9|1|3|7|8|6|4|5|2
6|8|2|5|9|4|7|3|1
3|6|8|4|7|9|2|1|5
5|4|1|2|6|3|8|9|7
2|9|7|1|5|8|3|6|4
   1) 
1|5|4|8|9|7|2|6|3
7|3|6|4|2|5|1|9|8
2|8|9|6|3|1|5|7|4
8|7|5|3|1|2|9|4|6
4|1|3|9|8|6|7|5|2
6|9|2|5|7|4|8|3|1
3|2|8|7|6|9|4|1|5
9|4|1|2|5|3|6|8|7
5|6|7|1|4|8|3|2|9
   2) 
1|5|4|8|9|7|6|2|3
7|3|6|4|2|5|1|9|8
2|8|9|6|3|1|5|7|4
8|7|5|3|1|2|9|4|6
4|1|3|9|8|6|7|5|2
6|9|2|7|5|4|8|3|1
3|6|8|2|7|9|4|1|5
9|4|1|5|6|3|2|8|7
5|2|7|1|4|8|3|6|9
   3) 
1|5|4|8|9|7|2|3|6
7|3|6|4|2|5|1|9|8
2|8|9|6|3|1|5|7|4
8|7|5|3|6|2|9|4|1
4|1|3|5|8|9|7|6|2
6|9|2|7|1|4|8|5|3
3|2|8|9|7|6|4|1|5
9|4|1|2|5|3|6|8|7
5|6|7|1|4|8|3|2|9
   4) 
1|5|4|8|9|7|2|3|6
7|3|6|4|2|5|1|9|8
2|8|9|6|3|1|5|7|4
8|7|5|3|6|2|9|4|1
4|1|3|7|8|9|6|5|2
6|9|2|5|1|4|7|8|3
3|2|8|9|7|6|4|1|5
9|4|1|2|5|3|8|6|7
5|6|7|1|4|8|3|2|9
   5) 
1|5|4|8|9|7|2|3|6
7|3|6|4|2|5|1|9|8
2|8|9|6|1|3|5|7|4
8|7|5|3|6|2|9|4|1
4|1|3|5|8|9|7|6|2
6|9|2|1|7|4|8|5|3
3|2|8|7|5|6|4|1|9
5|4|1|9|3|

## Generate some Sudoku Puzzles

Generate 5 filled in Sudoku boards with first row specified.

In [10]:
Results := Sudoku.Solve(5, "123456789");

Results.Boards->Count();
Results.Boards;

5
Seq<str>
   0) 
1|2|3|4|5|6|7|8|9
9|8|7|3|2|1|4|6|5
6|5|4|9|8|7|2|3|1
2|9|8|7|6|5|3|1|4
7|6|5|1|3|4|8|9|2
4|3|1|8|9|2|6|5|7
5|4|9|6|7|3|1|2|8
8|1|6|2|4|9|5|7|3
3|7|2|5|1|8|9|4|6
   1) 
1|2|3|4|5|6|7|8|9
9|7|5|2|1|8|3|6|4
6|8|4|3|9|7|5|2|1
5|9|8|6|2|1|4|7|3
7|4|6|9|8|3|2|1|5
2|3|1|7|4|5|6|9|8
4|5|9|8|7|2|1|3|6
3|1|7|5|6|9|8|4|2
8|6|2|1|3|4|9|5|7
   2) 
1|2|3|4|5|6|7|8|9
7|5|8|9|2|3|6|1|4
6|4|9|1|8|7|3|5|2
5|7|6|8|9|4|2|3|1
4|3|2|6|1|5|8|9|7
8|9|1|3|7|2|5|4|6
9|6|7|5|4|8|1|2|3
3|8|4|2|6|1|9|7|5
2|1|5|7|3|9|4|6|8
   3) 
1|2|3|4|5|6|7|8|9
8|4|9|3|2|7|6|1|5
7|5|6|9|1|8|4|2|3
3|9|8|7|6|1|2|5|4
6|7|2|5|8|4|3|9|1
5|1|4|2|9|3|8|6|7
9|3|5|6|4|2|1|7|8
2|8|7|1|3|5|9|4|6
4|6|1|8|7|9|5|3|2
   4) 
1|2|3|4|5|6|7|8|9
7|4|9|2|1|8|3|6|5
8|5|6|9|3|7|4|2|1
5|9|8|7|2|1|6|4|3
3|1|2|6|8|4|5|9|7
6|7|4|5|9|3|8|1|2
4|6|5|1|7|9|2|3|8
9|8|7|3|6|2|1|5|4
2|3|1|8|4|5|9|7|6


## Generate some Rank 4 Boards

The MIP solvers start to struggle when asked to generate higher rank boards.
The SAT solver has an easy time of it.

In [11]:
#!time
Results := Sudoku.Private.Core("1234567890ABCDEF", 4, 2);

Results.Boards->Count();
Results.Boards;

2
Seq<str>
   0) 
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
9|B|0|D|F|E|4|C|7|8|1|3|6|5|A|2
8|7|6|E|D|9|A|2|F|5|C|4|0|3|B|1
F|C|5|A|3|1|B|0|E|6|2|D|4|8|9|7
4|6|7|9|0|C|8|A|2|1|D|F|5|E|3|B
A|3|8|2|B|D|6|9|5|7|0|E|1|F|4|C
0|D|E|F|1|3|2|5|C|B|4|9|7|A|6|8
C|5|1|B|7|4|E|F|A|3|6|8|9|2|0|D
E|1|B|C|6|7|3|4|8|A|F|2|D|0|5|9
2|F|D|6|9|A|0|B|3|4|5|1|8|C|7|E
5|0|4|7|2|8|1|D|B|E|9|C|A|6|F|3
3|9|A|8|C|F|5|E|0|D|7|6|2|B|1|4
6|E|F|5|4|B|C|1|D|9|8|A|3|7|2|0
7|4|C|0|8|2|9|3|6|F|E|5|B|1|D|A
D|8|9|3|A|5|F|7|1|2|B|0|E|4|C|6
B|A|2|1|E|0|D|6|4|C|3|7|F|9|8|5
   1) 
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
B|7|8|0|A|9|3|E|D|2|F|C|4|5|6|1
5|E|6|F|C|2|0|D|4|8|1|3|B|9|7|A
9|A|C|D|4|1|B|F|7|E|5|6|8|2|0|3
D|9|5|A|F|C|1|6|E|3|2|7|0|B|8|4
4|F|E|B|7|8|9|3|0|D|6|A|1|C|2|5
0|6|7|1|D|4|2|5|C|9|B|8|3|A|F|E
8|3|2|C|E|0|A|B|5|1|4|F|D|6|9|7
E|B|4|7|9|F|6|1|8|A|D|0|2|3|5|C
F|1|D|2|3|A|E|C|6|4|7|5|9|8|B|0
C|5|A|8|2|7|D|0|F|B|3|9|E|1|4|6
6|0|9|3|8|B|5|4|2|C|E|1|7|F|A|D
7|D|F|9|1|3|8|A|B|5|0|4|6|E|C|2
3|4|1|6|B|E|C|7|A|F|8|2|5|0|D|9
2|8|0|E|6|5|F|9

Wall time: 108.6969ms

## Rank 5

The SAT solver starts to struggle with rank 5.

In [12]:
#!time
Results := Sudoku.Private.Core("1234567890ABCDEFGHIJKLMNO", 5, 2);

Results.Boards->Count();
Results.Boards;

2
Seq<str>
   0) 
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O
O|A|9|H|7|G|E|2|I|4|M|0|F|N|L|D|C|6|K|1|5|8|3|J|B
I|N|G|E|0|B|A|5|J|F|4|O|8|K|9|2|L|7|3|M|D|6|H|1|C
F|D|8|C|L|N|H|K|M|3|5|I|1|J|6|0|E|O|B|4|A|G|7|2|9
K|M|B|J|6|1|C|O|L|D|3|2|G|H|7|9|5|A|8|N|F|E|0|I|4
3|G|4|A|K|2|8|0|E|6|C|L|N|I|D|H|J|F|M|5|9|7|O|B|1
0|5|D|7|O|4|I|N|1|9|E|J|B|M|H|C|A|L|6|2|8|3|G|F|K
N|C|H|9|F|O|K|J|5|L|0|7|3|A|G|B|4|E|1|8|M|2|I|6|D
L|B|E|2|I|H|F|D|7|M|1|9|6|8|5|3|N|K|O|G|C|0|A|4|J
6|1|M|8|J|C|B|3|A|G|F|K|O|4|2|7|0|I|D|9|L|5|N|E|H
M|3|A|I|N|8|4|6|O|E|B|G|J|F|0|5|1|D|H|C|7|9|K|L|2
7|O|J|0|4|M|9|L|H|K|N|5|A|1|C|I|2|8|G|6|B|F|E|D|3
5|L|C|F|9|7|G|1|3|A|H|D|2|O|K|N|B|4|E|0|I|J|8|M|6
H|K|2|1|E|D|J|B|0|I|L|6|7|3|8|A|F|M|9|O|4|C|5|G|N
D|8|6|G|B|5|2|C|F|N|I|M|E|9|4|L|K|3|J|7|0|O|1|H|A
2|I|K|5|1|3|L|H|D|O|7|A|4|B|J|8|6|G|N|F|E|M|C|9|0
C|9|7|B|3|A|6|4|2|1|G|H|M|L|I|O|D|0|5|E|N|K|J|8|F
E|J|0|M|G|I|5|F|K|B|8|N|9|6|O|4|3|C|L|H|2|1|D|A|7
A|6|L|O|8|E|M|9|N|7|D|1|0|C|F|J|I|B|2|K|G|H|4|3|5
4|H|F|N|D|J|0|G|8|C|2|E|K|5|3|1|

Wall time: 831.9569ms

## Rank 6?

The SAT solver is overwhelmed by rank 6.